Used the library from here: https://github.com/alt-code/Research/tree/master/SimpleGenderComputer

Please use it with Python 2.7

In [2]:
import pickle
import pandas as pd
from simpleGenderComputer import SimpleGenderComputer

In [3]:
gc = SimpleGenderComputer('./namelists')

In [4]:
gc.simpleLookup('Ágnes')

'Unknown'

In [5]:
gc.simpleLookup('Agnes')

'Female'

In [6]:
def get_author_name(name):
    words = name.split()
    # must has first and last name to query the API
    if len(words) <= 1:
        payload = 'invalid'
    else:
        payload = {'Fname': words[0], 'Lname': words[-1]}
    return payload

### Authors in self-promotion data

In [5]:
! wc -l ./author_names.txt

 3081657 ./author_names.txt


In [6]:
name_gender = {}
with open('./author_names.txt', 'r') as ifile:
    for name in ifile:
        # name is already normalized
        # get rid of '\n' at the end of each line
        name = name.strip()
        name_tupe = get_author_name(name)
        if type(name_tupe) == dict:
            fname = name_tupe['Fname']
            fname = fname.decode('utf-8', 'ignore')
            gender = gc.simpleLookup(fname)
            name_gender[name] = gender

In [7]:
len(name_gender)

3066942

In [8]:
with open("./name_gender.txt", 'w') as ofile:
    for name, gender in name_gender.items():
        ofile.write(name + '||' + gender + '\n')

### Authors in IOP data

In [7]:
iop_clean_name_self_gender = pd.read_csv('./iop_clean_name_self_gender.csv')

In [9]:
def pred_gender_ford(name):
    if type(name) == str:
        name = name.strip()
        name_tupe = get_author_name(name)
        if type(name_tupe) == dict:
            fname = name_tupe['Fname']
            fname = fname.decode('utf-8', 'ignore')
            if len(fname) > 1:
                gender = gc.simpleLookup(fname)
                return gender
            else:
                return 'invalid'
        else:
            return 'invalid'
    else:
        return 'invalid'

In [10]:
iop_clean_name_self_gender['gender_ford'] = iop_clean_name_self_gender['clean_name'].apply(pred_gender_ford)

In [11]:
iop_clean_name_self_gender.head()

clean_name Task Country (at time of activity)  \
0         Satish Kumar                              India   
1  Priya Sachin Jadhav                              India   
2   Arunkumar  Bongale                              India   
3   Arunkumar  Bongale                              India   
4   Arunkumar  Bongale                              India   

  Contact Confirmed Gender gender_ford  
0                     Male        Male  
1                   Female      Female  
2                     Male        Male  
3                     Male        Male  
4                     Male        Male

In [12]:
iop_clean_name_self_gender.to_csv("./iop_clean_name_self_gender_ford.csv", index=False, header=True, encoding='utf-8')